In [6]:
from hdbcli import dbapi

# Define the database credentials
HC = {
    'endpoint': '9c272423-17ca-4bee-9c1f-6b46cc5d5340.hana.trial-us10.hanacloud.ondemand.com',
    'port': 443,  # SAP HANA uses port 443 for SSL connections
     "user": os.getenv('user'),
    "password": os.getenv('password')
}

# Function to establish connection
def get_connection():
    address = HC['endpoint'] 
    port = HC['port']
    user = HC['user']
    password = HC['password']
    
    try:
        conn = dbapi.connect(
            address=address,
            port=port,
            user=user,
            password=password,
            encrypt=True,
            sslValidateCertificate=False
        )
        print("Connection established")
        return conn
    except dbapi.Error as err:
        print(f"Error: {err}")
        return None

def execute_sql(sql):
    print(sql)
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    results = rows
    cursor_descriptions = cursor.description
    cursor.close()
    conn.close()
    return cursor_descriptions, results

def db_query(sql, outputformat = 'raw'):
    cursor_descriptions, rows = execute_sql(sql)
    return format_data(rows, cursor_descriptions, outputformat)

def format_data(rows, cursor_descriptions, output_format = 'raw'):
    results = []
    for row in rows:
        result = {}
        for index in range(len(row)):
            result[cursor_descriptions[index][0]] = row[index]
        results.append(result)
    ret = results
    if output_format == 'json':
        ret = json.dumps(results, cls=JSONEncoder)
    elif output_format == 'dataframe':
        ret = pd.DataFrame(results)
    return ret

In [19]:
# sql='''SELECT DISTINCT SCHEMA_NAME, TABLE_NAME, COLUMN_NAME, DATA_TYPE_NAME, LENGTH, SCALE FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = 'DATMOK';'''
# sql='''SELECT TABLE_NAME FROM SYS.TABLES WHERE SCHEMA_NAME = 'DATMOK';'''
# sql='''SELECT DepartmentID, COUNT(*) as "Number of Satisfactory Performances" FROM DATMOK.EmployeePerformance WHERE PERFORMANCE = 'Satisfactory' GROUP BY DepartmentID;'''
sql='''SELECT * FROM M_TABLES WHERE TABLE_NAME = 'EmployeeHeadcount' AND SCHEMA_NAME = 'DATMOK';'''
# sql='''SELECT COUNT(DISTINCT EmployeeID) AS NumberOfEmployeesFROM DATMOK.EmployeeHeadcount; '''
result = db_query(sql, 'raw')

SELECT * FROM M_TABLES WHERE TABLE_NAME = 'EmployeeHeadcount' AND SCHEMA_NAME = 'DATMOK';
Connection established


In [20]:
result

[{'SCHEMA_NAME': 'DATMOK',
  'TABLE_NAME': 'EmployeeHeadcount',
  'RECORD_COUNT': 1828,
  'TABLE_SIZE': 1344,
  'IS_COLUMN_TABLE': 'TRUE',
  'TABLE_TYPE': 'COLUMN',
  'IS_PARTITIONED': 'FALSE',
  'IS_REPLICATED': 'FALSE',
  'HAS_RECORD_COMMIT_TIMESTAMP': 'FALSE'}]